In [3]:
## load
# Filename: 01_intro_chatbot.ipynb

import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

# Load Groq API key from environment variable
groq_api_key = os.getenv("GROQ_API_KEY")


In [4]:
##llm model
# Import ChatGroq from langchain_groq
from langchain_groq import ChatGroq

# Create the chatbot model
model = ChatGroq(
    temperature=0.7,
    groq_api_key=groq_api_key,
    model="gemma2-9b-it"
)


In [5]:
## langchian message history 

from langchain_core.messages import HumanMessage

## simulate humam message 
model.invoke([
    HumanMessage(content="Hi, my name is ajay mokta. i am the founder of unisole")
])

AIMessage(content="Hi Ajay, it's nice to meet you! \n\nI'm excited to learn more about Unisole.  Could you tell me a little bit about it? What does Unisole do? What inspired you to start it?  \n\nI'm always interested in hearing about innovative new companies and the stories behind them.  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 26, 'total_tokens': 98, 'completion_time': 0.130909091, 'prompt_time': 0.001410019, 'queue_time': 0.270275001, 'total_time': 0.13231911}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--05189100-14b6-46d8-a392-d839a41fa1e3-0', usage_metadata={'input_tokens': 26, 'output_tokens': 72, 'total_tokens': 98})

In [7]:
## combining ai and human messages\
from langchain_core.messages import AIMessage
## provide history to understand proper flow
model.invoke([
    HumanMessage(content="hi, i am ajay mokta."),
    AIMessage(content="hello Ajay, how can i help you today"),
    HumanMessage(content="hey, what is my name?")
])

AIMessage(content='Your name is Ajay Mokta. 😊  You told me! \n\n\n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 44, 'total_tokens': 72, 'completion_time': 0.050909091, 'prompt_time': 0.001632699, 'queue_time': 0.273521431, 'total_time': 0.05254179}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--72e2bbe1-1b25-45ac-a5b8-ab7c31c71306-0', usage_metadata={'input_tokens': 44, 'output_tokens': 28, 'total_tokens': 72})

In [8]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [9]:
## store message history
store={}

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]



In [12]:
from langchain_core.runnables.history import RunnableWithMessageHistory

with_message_history = RunnableWithMessageHistory(
    model,
    get_session_history  # <- ✅ notice: no parentheses
)


In [13]:
## define confi session

config={
    "configurable":{
        "session_id":"chat_2"
    }
}

In [18]:
response = with_message_history.invoke(
    [HumanMessage(content="my name is ajay mokta?")],
    config=config
)
print(response.content)

Yes, you said your name is Ajay Mokta!  

Is there something specific you want to know or do with that information? 😊  I'm happy to help!  



In [16]:
response = with_message_history.invoke(
    [HumanMessage(content="what is my  name?")],
    config=config
)
print(response.content)

You told me your name is Ajay Mokta!  

Do you need me to remember that for later?  Perhaps you'd like to tell me something else about yourself?  😄 



In [22]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


In [23]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all the questions to the best of your ability."),
    MessagesPlaceholder(variable_name="input_history")
])


In [24]:
chain = prompt | model


In [25]:
chain.invoke([
    HumanMessage(content="Hi, Say my name")
])


AIMessage(content='Please tell me your name so I can say it! 😊  \n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 31, 'total_tokens': 48, 'completion_time': 0.030909091, 'prompt_time': 0.00148221, 'queue_time': 0.263029801, 'total_time': 0.032391301}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--688b0d92-f02e-4d42-a3d7-b34b40c50e1d-0', usage_metadata={'input_tokens': 31, 'output_tokens': 17, 'total_tokens': 48})

In [26]:
config_chat2 = {
    "configurable": {
        "session_id": "chat_2" # Or whatever session ID you introduced yourself in
    }
}

In [27]:
response1 = with_message_history.invoke(
    [HumanMessage(content="Hi, I am Ajay Mokta, founder of Unisole.")],
    config=config_chat2
)

In [28]:
print(f"AI (1st turn): {response1.content}")

AI (1st turn): Hello Ajay Mokta, founder of Unisole! It's great to meet you. 

What can I do for you today? Are you looking for information about Unisole, or do you have a specific question for me? 







In [29]:
response2 = with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config_chat2 # using the SAME session_id
)

In [30]:
print(f"AI (2nd turn): {response2.content}")

AI (2nd turn): You said your name is Ajay Mokta.  😊  

Is there anything else I can help you with?



In [31]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history  # previously defined function
)


In [36]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all questions to the best of your ability in {language}."), 
    MessagesPlaceholder(variable_name="input_history") 
])

In [37]:
chain = prompt | model


In [39]:
response = chain.invoke(
    {
        
        "input_history": [HumanMessage(content="Hi Ajay Mokta from Himachal")],
        "language": "Hindi"
    }
)
print(response.content)

नमस्ते अजय मोक्ता जी ! हिमाचल से आपका स्वागत है! 

मुझे बताइए, मैं आपके लिए कैसे मदद कर सकता हूँ? 😊




In [40]:
config = {
    "configurable": {
        "session_id": "chat4"
    }
}
